# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [2]:
# Load the data as aviation_df
aviation_df = pd.read_csv(
    "data/AviationData.csv",
    encoding="latin1",
    low_memory=False
)

aviation_shape = aviation_df.shape
aviation_shape

(88889, 31)

In [3]:
aviation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

In [4]:
for col in aviation_df.columns:
    print(col)

Event.Id
Investigation.Type
Accident.Number
Event.Date
Location
Country
Latitude
Longitude
Airport.Code
Airport.Name
Injury.Severity
Aircraft.damage
Aircraft.Category
Registration.Number
Make
Model
Amateur.Built
Number.of.Engines
Engine.Type
FAR.Description
Schedule
Purpose.of.flight
Air.carrier
Total.Fatal.Injuries
Total.Serious.Injuries
Total.Minor.Injuries
Total.Uninjured
Weather.Condition
Broad.phase.of.flight
Report.Status
Publication.Date


In [5]:
pd.set_option('display.max_columns', None)
aviation_df.tail()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,Aircraft.Category,Registration.Number,Make,Model,Amateur.Built,Number.of.Engines,Engine.Type,FAR.Description,Schedule,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
88884,20221227106491,Accident,ERA23LA093,2022-12-26,"Annapolis, MD",United States,NaN,NaN,NaN,NaN,Minor,NaN,NaN,N1867H,PIPER,PA-28-151,No,NaN,NaN,091,NaN,Personal,NaN,0.0,1.0,0.0,0.0,NaN,NaN,NaN,29-12-2022
88885,20221227106494,Accident,ERA23LA095,2022-12-26,"Hampton, NH",United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N2895Z,BELLANCA,7ECA,No,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
88886,20221227106497,Accident,WPR23LA075,2022-12-26,"Payson, AZ",United States,341525N,1112021W,PAN,PAYSON,Non-Fatal,Substantial,Airplane,N749PJ,AMERICAN CHAMPION AIRCRAFT,8GCBC,No,1.0,NaN,091,NaN,Personal,NaN,0.0,0.0,0.0,1.0,VMC,NaN,NaN,27-12-2022
88887,20221227106498,Accident,WPR23LA076,2022-12-26,"Morgan, UT",United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N210CU,CESSNA,210N,No,NaN,NaN,091,NaN,Personal,MC CESSNA 210N LLC,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
88888,20221230106513,Accident,ERA23LA097,2022-12-29,"Athens, GA",United States,NaN,NaN,NaN,NaN,Minor,NaN,NaN,N9026P,PIPER,PA-24-260,No,NaN,NaN,091,NaN,Personal,NaN,0.0,1.0,0.0,1.0,NaN,NaN,NaN,30-12-2022


In [6]:
aviation_df.isna().mean().sort_values(ascending=False)

Schedule                  0.858453
Air.carrier               0.812710
FAR.Description           0.639742
Aircraft.Category         0.636772
Longitude                 0.613304
Latitude                  0.613203
Airport.Code              0.436016
Airport.Name              0.407081
Broad.phase.of.flight     0.305606
Publication.Date          0.154924
Total.Serious.Injuries    0.140737
Total.Minor.Injuries      0.134246
Total.Fatal.Injuries      0.128261
Engine.Type               0.079830
Report.Status             0.071820
Purpose.of.flight         0.069660
Number.of.Engines         0.068445
Total.Uninjured           0.066510
Weather.Condition         0.050535
Aircraft.damage           0.035932
Registration.Number       0.015547
Injury.Severity           0.011250
Country                   0.002542
Amateur.Built             0.001147
Model                     0.001035
Make                      0.000709
Location                  0.000585
Investigation.Type        0.000000
Event.Date          

## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [7]:

#Convert Event.Date to datetime
aviation_df['Event.Date'] = pd.to_datetime(
    aviation_df['Event.Date'], errors="coerce"
)

In [8]:
#Add an Event.Year column derived from Event.Date
aviation_df['Event.Year'] = aviation_df['Event.Date'].dt.year
aviation_df.head() 

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,Aircraft.Category,Registration.Number,Make,Model,Amateur.Built,Number.of.Engines,Engine.Type,FAR.Description,Schedule,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Event.Year
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,Fatal(2),Destroyed,NaN,NC6404,Stinson,108-3,No,1.0,Reciprocating,NaN,NaN,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN,1948
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,Fatal(4),Destroyed,NaN,N5069P,Piper,PA24-180,No,1.0,Reciprocating,NaN,NaN,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996,1962
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,Fatal(3),Destroyed,NaN,N5142R,Cessna,172M,No,1.0,Reciprocating,NaN,NaN,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007,1974
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,Fatal(2),Destroyed,NaN,N1168J,Rockwell,112,No,1.0,Reciprocating,NaN,NaN,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000,1977
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,Fatal(1),Destroyed,NaN,N15NY,Cessna,501,No,NaN,NaN,NaN,NaN,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980,1979


In [9]:
#Filter out events before 1983
aviation_filter_year_df = aviation_df[aviation_df['Event.Year'] >= 1983]
aviation_filter_year_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85289 entries, 3600 to 88888
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                85289 non-null  object        
 1   Investigation.Type      85289 non-null  object        
 2   Accident.Number         85289 non-null  object        
 3   Event.Date              85289 non-null  datetime64[ns]
 4   Location                85237 non-null  object        
 5   Country                 85073 non-null  object        
 6   Latitude                34379 non-null  object        
 7   Longitude               34370 non-null  object        
 8   Airport.Code            48435 non-null  object        
 9   Airport.Name            50514 non-null  object        
 10  Injury.Severity         84289 non-null  object        
 11  Aircraft.damage         82151 non-null  object        
 12  Aircraft.Category       28723 non-null  object  

In [10]:
#Filter out Amateur Built aircrafts
aviation_filtered_df = aviation_filter_year_df[aviation_filter_year_df['Amateur.Built'] == "No"].copy()
aviation_filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76960 entries, 3600 to 88888
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                76960 non-null  object        
 1   Investigation.Type      76960 non-null  object        
 2   Accident.Number         76960 non-null  object        
 3   Event.Date              76960 non-null  datetime64[ns]
 4   Location                76913 non-null  object        
 5   Country                 76750 non-null  object        
 6   Latitude                30167 non-null  object        
 7   Longitude               30161 non-null  object        
 8   Airport.Code            43375 non-null  object        
 9   Airport.Name            45285 non-null  object        
 10  Injury.Severity         75961 non-null  object        
 11  Aircraft.damage         73868 non-null  object        
 12  Aircraft.Category       25405 non-null  object  

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [11]:
#Fill missing injury counts
#Assumption: If Total.Fatal.Injuries is missing, it likely means 0 injuries reported
aviation_filtered_df["Total.Fatal.Injuries"] = aviation_filtered_df["Total.Fatal.Injuries"].fillna(0)
aviation_filtered_df["Total.Serious.Injuries"] = aviation_filtered_df["Total.Serious.Injuries"].fillna(0)
aviation_filtered_df["Total.Minor.Injuries"] = aviation_filtered_df["Total.Minor.Injuries"].fillna(0)
aviation_filtered_df["Total.Uninjured"] = aviation_filtered_df["Total.Uninjured"].fillna(0)

aviation_filtered_df[['Event.Id','Total.Fatal.Injuries','Total.Serious.Injuries','Total.Minor.Injuries','Total.Uninjured']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 76960 entries, 3600 to 88888
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                76960 non-null  object 
 1   Total.Fatal.Injuries    76960 non-null  float64
 2   Total.Serious.Injuries  76960 non-null  float64
 3   Total.Minor.Injuries    76960 non-null  float64
 4   Total.Uninjured         76960 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.5+ MB


In [12]:
#Estimate total passengers on each flight
#Total Passengers = Fatal + Serious + Minor + Uninjured
aviation_filtered_df['Total.Passengers'] = (
    aviation_filtered_df['Total.Fatal.Injuries'] +
    aviation_filtered_df['Total.Serious.Injuries'] +
    aviation_filtered_df['Total.Minor.Injuries'] +
    aviation_filtered_df['Total.Uninjured']
)

aviation_filtered_df.tail(10)

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,Aircraft.Category,Registration.Number,Make,Model,Amateur.Built,Number.of.Engines,Engine.Type,FAR.Description,Schedule,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Event.Year,Total.Passengers
88879,20221219106472,Accident,DCA23LA096,2022-12-18,"Kahului, HI",United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N393HA,AIRBUS,A330-243,No,NaN,NaN,121,SCHD,NaN,HAWAIIAN AIRLINES INC,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2022,0.0
88880,20221219106477,Accident,WPR23LA071,2022-12-18,"San Manual, AZ",United States,NaN,NaN,NaN,NaN,Non-Fatal,NaN,NaN,N4144P,PIPER,PA28,No,NaN,NaN,091,NaN,Personal,Chandler Air Service,0.0,0.0,0.0,3.0,NaN,NaN,NaN,20-12-2022,2022,3.0
88881,20221221106483,Accident,CEN23LA067,2022-12-21,"Auburn Hills, MI",United States,NaN,NaN,NaN,NaN,Minor,NaN,NaN,N8786U,CESSNA,172F,No,NaN,NaN,091,NSCH,Personal,Pilot,0.0,1.0,0.0,0.0,NaN,NaN,NaN,22-12-2022,2022,1.0
88882,20221222106486,Accident,CEN23LA068,2022-12-21,"Reserve, LA",United States,NaN,NaN,NaN,NaN,Minor,NaN,NaN,N321GD,GRUMMAN AMERICAN AVN. CORP.,AA-5B,No,NaN,NaN,091,NaN,Instructional,NaN,0.0,1.0,0.0,1.0,NaN,NaN,NaN,27-12-2022,2022,2.0
88883,20221228106502,Accident,GAA23WA046,2022-12-22,"Brasnorte,",Brazil,NaN,NaN,NaN,NaN,Fatal,NaN,NaN,PP-IRC,AIR TRACTOR,AT502,No,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,NaN,NaN,NaN,28-12-2022,2022,1.0
88884,20221227106491,Accident,ERA23LA093,2022-12-26,"Annapolis, MD",United States,NaN,NaN,NaN,NaN,Minor,NaN,NaN,N1867H,PIPER,PA-28-151,No,NaN,NaN,091,NaN,Personal,NaN,0.0,1.0,0.0,0.0,NaN,NaN,NaN,29-12-2022,2022,1.0
88885,20221227106494,Accident,ERA23LA095,2022-12-26,"Hampton, NH",United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N2895Z,BELLANCA,7ECA,No,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2022,0.0
88886,20221227106497,Accident,WPR23LA075,2022-12-26,"Payson, AZ",United States,341525N,1112021W,PAN,PAYSON,Non-Fatal,Substantial,Airplane,N749PJ,AMERICAN CHAMPION AIRCRAFT,8GCBC,No,1.0,NaN,091,NaN,Personal,NaN,0.0,0.0,0.0,1.0,VMC,NaN,NaN,27-12-2022,2022,1.0
88887,20221227106498,Accident,WPR23LA076,2022-12-26,"Morgan, UT",United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N210CU,CESSNA,210N,No,NaN,NaN,091,NaN,Personal,MC CESSNA 210N LLC,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2022,0.0
88888,20221230106513,Accident,ERA23LA097,2022-12-29,"Athens, GA",United States,NaN,NaN,NaN,NaN,Minor,NaN,NaN,N9026P,PIPER,PA-24-260,No,NaN,NaN,091,NaN,Personal,NaN,0.0,1.0,0.0,1.0,NaN,NaN,NaN,30-12-2022,2022,2.0


In [13]:
#Compute the fraction of passengers fatally or seriously injured
#Assumption: If Total.Passengers = 0, we set the fraction to NaN to avoid divide-by-zero
aviation_filtered_df['Frac.Fatal.Serious'] = (
    (aviation_filtered_df['Total.Fatal.Injuries'] + aviation_filtered_df['Total.Serious.Injuries']) /
    aviation_filtered_df['Total.Passengers']
).replace([np.inf, -np.inf], np.nan)
aviation_filtered_df[['Event.Id','Total.Fatal.Injuries','Total.Serious.Injuries','Total.Minor.Injuries','Total.Uninjured','Total.Passengers','Frac.Fatal.Serious']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 76960 entries, 3600 to 88888
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                76960 non-null  object 
 1   Total.Fatal.Injuries    76960 non-null  float64
 2   Total.Serious.Injuries  76960 non-null  float64
 3   Total.Minor.Injuries    76960 non-null  float64
 4   Total.Uninjured         76960 non-null  float64
 5   Total.Passengers        76960 non-null  float64
 6   Frac.Fatal.Serious      75701 non-null  float64
dtypes: float64(6), object(1)
memory usage: 4.7+ MB


In [14]:
## Create binary flags for fata, serious, or destroyed
aviation_filtered_df['fatal_accident'] = aviation_filtered_df['Total.Fatal.Injuries'] > 0
aviation_filtered_df['serious_accident'] = aviation_filtered_df['Total.Serious.Injuries'] > 0
aviation_filtered_df['fatal_or_serious'] = aviation_filtered_df['fatal_accident'] | aviation_filtered_df['serious_accident']

aviation_filtered_df[[
    "Total.Passengers", "Total.Fatal.Injuries",
    "Total.Serious.Injuries", "Frac.Fatal.Serious", "fatal_or_serious"
]].head(10)

,Total.Passengers,Total.Fatal.Injuries,Total.Serious.Injuries,Frac.Fatal.Serious,fatal_or_serious
3600,2.0,0.0,1.0,0.5,True
3601,4.0,0.0,0.0,0.0,False
3602,2.0,0.0,0.0,0.0,False
3603,1.0,0.0,0.0,0.0,False
3604,2.0,0.0,0.0,0.0,False
3605,2.0,0.0,0.0,0.0,False
3606,2.0,0.0,1.0,0.5,True
3607,4.0,0.0,0.0,0.0,False
3608,2.0,2.0,0.0,1.0,True
3609,3.0,3.0,0.0,1.0,True


**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [15]:
# See unique values in column
aviation_filtered_df['Aircraft.damage'].value_counts(dropna=False)

Aircraft.damage
Substantial    55689
Destroyed      15469
NaN             3092
Minor           2594
Unknown          116
Name: count, dtype: int64

In [16]:
# Fill NaN to 'Unknown'
aviation_filtered_df['Aircraft.damage'] = aviation_filtered_df['Aircraft.damage'].fillna('Unknown')
aviation_filtered_df['Aircraft.damage'].value_counts(dropna=False)

Aircraft.damage
Substantial    55689
Destroyed      15469
Unknown         3208
Minor           2594
Name: count, dtype: int64

In [17]:
## Create a binary flag: True if Destroyed, False otherwise
aviation_filtered_df['destroyed'] = aviation_filtered_df['Aircraft.damage'] == 'Destroyed'
aviation_filtered_df[['Aircraft.damage','destroyed']].value_counts()

Aircraft.damage  destroyed
Substantial      False        55689
Destroyed        True         15469
Unknown          False         3208
Minor            False         2594
Name: count, dtype: int64

### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

### Inspect the Make column
- Look for:
    - Typos / inconsistent capitalization
    - Missing values (NaN)
    - Rare makes with very few accidents

In [18]:
#Check unique makes and top occurences
aviation_filtered_df['Make'].value_counts(dropna=False).head(20)

Make
Cessna               20818
Piper                11260
CESSNA                4922
Beech                 4052
PIPER                 2839
Bell                  2006
Boeing                1498
BOEING                1151
BEECH                 1042
Mooney                1029
Grumman                987
Robinson               914
Bellanca               819
Hughes                 740
Schweizer              609
BELL                   588
Air Tractor            577
Mcdonnell Douglas      511
Aeronca                458
Maule                  427
Name: count, dtype: int64

### Identify Cleaning Tasks
Proposed cleaning tasks:
1. Standardize capitalization
    - Convert all entries to title case (boeing to Boeing)
2. Remove missing values
    - Remove rows where Make is NaN since we cannot assign safety metrics to unknown aircraft
3. Remove rare Makes
    - Deep only Makes with >= 50 occurrences to ensure statistical robustness

In [19]:
# Standardize capitalization
aviation_filtered_df['Make'] = aviation_filtered_df['Make'].str.title()
aviation_filtered_df['Make'].value_counts().head(30)

Make
Cessna               25740
Piper                14099
Beech                 5094
Boeing                2649
Bell                  2594
Mooney                1271
Robinson              1197
Grumman               1064
Bellanca               978
Hughes                 877
Schweizer              753
Air Tractor            673
Aeronca                607
Mcdonnell Douglas      593
Maule                  571
Champion               504
Stinson                421
Aero Commander         411
De Havilland           405
Luscombe               390
North American         370
Taylorcraft            366
Aerospatiale           360
Rockwell               337
Hiller                 329
Airbus                 289
Enstrom                281
Douglas                262
Embraer                236
Ayres                  231
Name: count, dtype: int64

In [20]:
# Drop rows with missing Make
aviation_filtered_df = aviation_filtered_df.dropna(subset=['Make'])
aviation_filtered_df[['Event.Id','Make']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 76914 entries, 3600 to 88888
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Event.Id  76914 non-null  object
 1   Make      76914 non-null  object
dtypes: object(2)
memory usage: 1.8+ MB


In [23]:
# Filter to makes with at least 50 occurences
make_counts = aviation_filtered_df['Make'].value_counts()
valid_makes = make_counts[make_counts >= 50].index
aviation_filtered_make_df = aviation_filtered_df[aviation_filtered_df['Make'].isin(valid_makes)].copy()
aviation_filtered_make_df['Make'].value_counts()

Make
Cessna                         25740
Piper                          14099
Beech                           5094
Boeing                          2649
Bell                            2594
                               ...  
American Champion Aircraft        54
Smith, Ted Aerostar               51
Flight Design Gmbh                50
Grumman American Avn. Corp.       50
Boeing Stearman                   50
Name: count, Length: 90, dtype: int64

In [24]:
# Mapping dictionary for Make standardization
make_mapping = {
    "Robinson Helicopter": "Robinson",
    "Robinson Helicopter Company": "Robinson",
    "Robinson": "Robinson",
    "Airbus Industrie": "Airbus",
    "Cirrus Design Corp": "Cirrus",
    "Cirrus Design Corp.": "Cirrus",
    "Cirrus": "Cirrus",
    "McDonnell Douglas": "McDonnell Douglas",
    "Douglas": "McDonnell Douglas",
    "Dehavilland": "De Havilland",
    "De Havilland": "De Havilland",
    "Grumman": "Grumman",
    "Grumman American": "Grumman",
    "Grumman American Avn. Corp.": "Grumman",
    "Boeing Stearman": "Boeing",
    "Aerospatiale": "Aerospatiale",
    "Air Tractor Inc": "Air Tractor",
    "Ercoupe (Eng & Research Corp.)": "Ercoupe",
    "Rockwell International": "Rockwell",
    "Raytheon Aircraft Company": "Raytheon Aircraft"
}
aviation_filtered_make_df['Make'] = aviation_filtered_make_df['Make'].replace(make_mapping)

In [32]:
make_counts_2 = aviation_filtered_make_df['Make'].value_counts()
valid_makes_2 = make_counts_2.index
valid_makes_2.sort_values()

Index(['Aero Commander', 'Aeronca', 'Aerospatiale', 'Aerostar', 'Agusta',
       'Air Tractor', 'Airbus', 'American', 'American Champion Aircraft',
       'Aviat', 'Aviat Aircraft Inc', 'Ayres', 'Balloon Works', 'Beech',
       'Bell', 'Bellanca', 'Boeing', 'Bombardier', 'Bombardier Inc',
       'British Aerospace', 'Burkhart Grob', 'Cameron', 'Canadair', 'Cessna',
       'Champion', 'Cirrus', 'De Havilland', 'Diamond Aircraft Ind Inc',
       'Embraer', 'Enstrom', 'Ercoupe', 'Eurocopter', 'Fairchild',
       'Flight Design Gmbh', 'Fokker', 'Gates Learjet', 'Globe', 'Great Lakes',
       'Grumman', 'Grumman Acft Eng Cor-Schweizer', 'Grumman-Schweizer',
       'Gulfstream', 'Helio', 'Hiller', 'Hughes', 'Lake', 'Learjet', 'Let',
       'Lockheed', 'Luscombe', 'Maule', 'Mbb', 'McDonnell Douglas',
       'Mcdonnell Douglas', 'Mitsubishi', 'Mooney', 'Navion', 'North American',
       'Pilatus', 'Piper', 'Pitts', 'Raven', 'Raytheon Aircraft', 'Robinson',
       'Rockwell', 'Ryan', 'Schempp-H

In [34]:
make_mapping_update = {
    "Mcdonnell Douglas": "McDonnell Douglas",
    "Grumman-Schweizer": "Grumman",
    "Grumman Acft Eng Cor-Schweizer": "Grumman",
    "Aviat Aircraft Inc": "Aviat",
    "Bombardier Inc": "Bombardier",
    "Smith, Ted Aerostar": "Aerostar",
    "Flight Design Gmbh": "Flight Design",
    "Let": "Let Aircraft"
}

aviation_filtered_make_df['Make'] = aviation_filtered_make_df['Make'].replace(make_mapping_update)

aviation_filtered_make_df['Make'].sort_values().unique()

array(['Aero Commander', 'Aeronca', 'Aerospatiale', 'Aerostar', 'Agusta',
       'Air Tractor', 'Airbus', 'American', 'American Champion Aircraft',
       'Aviat', 'Ayres', 'Balloon Works', 'Beech', 'Bell', 'Bellanca',
       'Boeing', 'Bombardier', 'British Aerospace', 'Burkhart Grob',
       'Cameron', 'Canadair', 'Cessna', 'Champion', 'Cirrus',
       'De Havilland', 'Diamond Aircraft Ind Inc', 'Embraer', 'Enstrom',
       'Ercoupe', 'Eurocopter', 'Fairchild', 'Flight Design', 'Fokker',
       'Gates Learjet', 'Globe', 'Great Lakes', 'Grumman', 'Gulfstream',
       'Helio', 'Hiller', 'Hughes', 'Lake', 'Learjet', 'Let Aircraft',
       'Lockheed', 'Luscombe', 'Maule', 'Mbb', 'McDonnell Douglas',
       'Mitsubishi', 'Mooney', 'Navion', 'North American', 'Pilatus',
       'Piper', 'Pitts', 'Raven', 'Raytheon Aircraft', 'Robinson',
       'Rockwell', 'Ryan', 'Schempp-Hirth', 'Schleicher', 'Schweizer',
       'Sikorsky', 'Socata', 'Stinson', 'Swearingen', 'Taylorcraft',
       'Waco', '

In [38]:
aviation_filtered_make_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70474 entries, 3601 to 88888
Data columns (total 38 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                70474 non-null  object        
 1   Investigation.Type      70474 non-null  object        
 2   Accident.Number         70474 non-null  object        
 3   Event.Date              70474 non-null  datetime64[ns]
 4   Location                70429 non-null  object        
 5   Country                 70280 non-null  object        
 6   Latitude                25964 non-null  object        
 7   Longitude               25959 non-null  object        
 8   Airport.Code            39817 non-null  object        
 9   Airport.Name            41672 non-null  object        
 10  Injury.Severity         69590 non-null  object        
 11  Aircraft.damage         70474 non-null  object        
 12  Aircraft.Category       21549 non-null  object  

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [39]:
aviation_filtered_make_df['Model'].isna().sum()

22

In [40]:
# Drop rows where Model is NaN (only 22)
aviation_filtered_make_df = aviation_filtered_make_df.dropna(subset=['Model'])

aviation_filtered_make_df['Model'].isna().sum()

0

In [41]:
# Count top 20 most common models
aviation_filtered_make_df['Model'].value_counts().head(20)

Model
152          2229
172          1649
172N         1093
PA-28-140     863
172M          759
150           752
172P          665
182           617
180           596
150M          551
PA-18         550
PA-18-150     549
PA-28-180     547
PA-28-161     523
PA-28-181     509
206B          488
737           487
150L          435
A36           428
PA-38-112     424
Name: count, dtype: int64

In [43]:
# Number of unique Makes per Model
model_make_counts = aviation_filtered_make_df.groupby('Model')['Make'].nunique()

# Models that appear more than one Make
non_unique_models = model_make_counts[model_make_counts > 1]
non_unique_models

Model
100           4
100-180       2
105           2
109           2
109A          2
             ..
TB-21         2
TB-9          2
TB20          2
TB21          2
THRUSH S2R    3
Name: Make, Length: 314, dtype: int64

In [45]:
# Create unique identified for plate type.
# Combined Make + Model to create a unique plane type.
aviation_filtered_make_df['PlaneType'] = (
    aviation_filtered_make_df['Make'] + aviation_filtered_make_df['Model']
)

aviation_filtered_make_df[['Make', 'Model', 'PlaneType']].head(10)

,Make,Model,PlaneType
3601,Cessna,182P,Cessna182P
3602,Cessna,182RG,Cessna182RG
3603,Cessna,182P,Cessna182P
3604,Piper,PA-28R-200,PiperPA-28R-200
3605,Cessna,140,Cessna140
3606,Balloon Works,FIREFLY 7B,Balloon WorksFIREFLY 7B
3607,Cessna,340A,Cessna340A
3608,North American,T-6G,North AmericanT-6G
3609,Piper,PA-24-250,PiperPA-24-250
3610,Piper,PA-32-301R,PiperPA-32-301R


In [49]:
aviation_filtered_make_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70452 entries, 3601 to 88888
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                70452 non-null  object        
 1   Investigation.Type      70452 non-null  object        
 2   Accident.Number         70452 non-null  object        
 3   Event.Date              70452 non-null  datetime64[ns]
 4   Location                70407 non-null  object        
 5   Country                 70258 non-null  object        
 6   Latitude                25954 non-null  object        
 7   Longitude               25949 non-null  object        
 8   Airport.Code            39809 non-null  object        
 9   Airport.Name            41660 non-null  object        
 10  Injury.Severity         69572 non-null  object        
 11  Aircraft.damage         70452 non-null  object        
 12  Aircraft.Category       21533 non-null  object  

### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

### Clean Engine.Type

In [64]:
# Work on a copy of the filtered dataframe
aviation_clean_df = aviation_filtered_make_df.copy()

In [65]:
# Show all unique Engine.Type values
aviation_clean_df['Engine.Type'].value_counts(dropna=False)

Engine.Type
Reciprocating      55006
NaN                 5417
Turbo Shaft         2989
Turbo Prop          2796
Turbo Fan           2173
Unknown             1513
Turbo Jet            543
Geared Turbofan       12
LR                     1
UNK                    1
NONE                   1
Name: count, dtype: int64

In [66]:
engine_mapping = {
    'Reciprocating': 'Reciprocating',
    'Turbo Shaft': 'Turboshaft',
    'Turbo Prop': 'Turboprop',
    'Turbo Fan': 'Turbofan',
    'Turbo Jet': 'Turbojet',
    'Geared Turbofan': 'Turbofan',
    'Unknown': 'Unknown',
    'UNK': 'Unknown',   # now Unknown
    'LR': 'Unknown',    # now Unknown
    'NONE': 'Unknown'   # now Unknown
}

# Apply mapping
aviation_clean_df['Engine.Type.Clean'] = aviation_clean_df['Engine.Type'].replace(engine_mapping)

# Fill any remaining NaNs with 'Unknown
aviation_clean_df['Engine.Type.Clean'] = aviation_clean_df['Engine.Type.Clean'].fillna('Unknown')

aviation_clean_df['Engine.Type.Clean'].value_counts()

Engine.Type.Clean
Reciprocating    55006
Unknown           6933
Turboshaft        2989
Turboprop         2796
Turbofan          2185
Turbojet           543
Name: count, dtype: int64

### Clean Weather.Condition

In [67]:
# Show all unique Weather.Condition values and counts
aviation_clean_df['Weather.Condition'].value_counts(dropna=False)

Weather.Condition
VMC    60560
IMC     5260
NaN     3699
UNK      704
Unk      229
Name: count, dtype: int64

In [68]:
weather_mapping = {
    'VMC': 'VMC',
    'IMC': 'IMC',
    'UNK': 'Unknown',
    'Unk': 'Unknown'
}

# Apply mapping and fill NaNs with 'Unknown
aviation_clean_df['Weather.Condition.Clean'] = aviation_clean_df['Weather.Condition'].replace(weather_mapping)
aviation_clean_df['Weather.Condition.Clean'] = aviation_clean_df['Weather.Condition.Clean'].fillna('Unknown')

aviation_clean_df['Weather.Condition.Clean'].value_counts()

Weather.Condition.Clean
VMC        60560
IMC         5260
Unknown     4632
Name: count, dtype: int64

### Number.Of.Engines

In [69]:
# Show unique values and counts
aviation_clean_df['Number.of.Engines'].value_counts(dropna=False)

Number.of.Engines
1.0    54450
2.0     9607
NaN     4821
0.0      741
3.0      431
4.0      401
8.0        1
Name: count, dtype: int64

### Purpose.of.flight

In [70]:
# Show all unique values and counts
aviation_clean_df['Purpose.of.flight'].value_counts(dropna=False)

Purpose.of.flight
Personal                     36907
Instructional                 9392
Unknown                       5504
NaN                           5328
Aerial Application            4134
Business                      3430
Positioning                   1446
Other Work Use                1061
Aerial Observation             708
Public Aircraft                643
Ferry                          640
Executive/corporate            424
Skydiving                      175
Flight Test                    158
Banner Tow                      97
External Load                   85
Public Aircraft - Federal       75
Public Aircraft - State         57
Public Aircraft - Local         52
Glider Tow                      41
Firefighting                    33
Air Race show                   27
Air Race/show                   18
Air Drop                        10
PUBS                             3
ASHO                             3
PUBL                             1
Name: count, dtype: int64

In [72]:
# Create a new column based on Purpose.of.flight
aviation_clean_df['Purpose.of.flight.Clean'] = aviation_clean_df['Purpose.of.flight'].replace({
    'Air Race/show': 'Air Race show'
})

# Fill NaNs with string 'NaN'
aviation_clean_df['Purpose.of.flight.Clean'] = aviation_clean_df['Purpose.of.flight.Clean'].fillna('NaN')

aviation_clean_df['Purpose.of.flight.Clean'].value_counts()

Purpose.of.flight.Clean
Personal                     36907
Instructional                 9392
Unknown                       5504
NaN                           5328
Aerial Application            4134
Business                      3430
Positioning                   1446
Other Work Use                1061
Aerial Observation             708
Public Aircraft                643
Ferry                          640
Executive/corporate            424
Skydiving                      175
Flight Test                    158
Banner Tow                      97
External Load                   85
Public Aircraft - Federal       75
Public Aircraft - State         57
Public Aircraft - Local         52
Air Race show                   45
Glider Tow                      41
Firefighting                    33
Air Drop                        10
PUBS                             3
ASHO                             3
PUBL                             1
Name: count, dtype: int64

### Broad.phase.of.flight

In [73]:
# Show unique values and counts
aviation_clean_df['Broad.phase.of.flight'].value_counts(dropna=False)

Broad.phase.of.flight
NaN            20402
Landing        13008
Takeoff         9799
Cruise          8397
Maneuvering     6265
Approach        5206
Taxi            1670
Climb           1656
Descent         1561
Go-around       1185
Standing         830
Unknown          394
Other             79
Name: count, dtype: int64

In [74]:
# Minimal cleaning with a new column
aviation_clean_df['Phase.Clean'] = aviation_clean_df['Broad.phase.of.flight'].replace({
    'Approach': 'Landing',
    'Go-around': 'Landing',
    'Take-off Run': 'Takeoff',
    'Takeoff Run': 'Takeoff',
    'Initial climb': 'Climb',
    'Climb-out': 'Climb'
})

# Fill NaNs with string 'NaN'
aviation_clean_df['Phase.Clean'] = aviation_clean_df['Phase.Clean'].fillna('NaN')

# Check result
aviation_clean_df['Phase.Clean'].value_counts()

Phase.Clean
NaN            20402
Landing        19399
Takeoff         9799
Cruise          8397
Maneuvering     6265
Taxi            1670
Climb           1656
Descent         1561
Standing         830
Unknown          394
Other             79
Name: count, dtype: int64

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [75]:
# Show fraction of missing values per column, sorted descending
aviation_clean_df.isna().mean().sort_values(ascending=False)

Schedule                   0.846491
Air.carrier                0.811773
FAR.Description            0.697099
Aircraft.Category          0.694359
Longitude                  0.631678
Latitude                   0.631607
Airport.Code               0.434949
Airport.Name               0.408675
Broad.phase.of.flight      0.289587
Publication.Date           0.172997
Engine.Type                0.076889
Purpose.of.flight          0.075626
Report.Status              0.071708
Number.of.Engines          0.068430
Weather.Condition          0.052504
Frac.Fatal.Serious         0.015684
Registration.Number        0.015500
Injury.Severity            0.012491
Country                    0.002754
Location                   0.000639
Purpose.of.flight.Clean    0.000000
Event.Year                 0.000000
Total.Passengers           0.000000
serious_accident           0.000000
fatal_accident             0.000000
Weather.Condition.Clean    0.000000
PlaneType                  0.000000
fatal_or_serious           0

In [76]:
# Drop very sparse columns (>50% missing)
cols_to_drop = ['Schedule', 'Air.carrier', 'FAR.Description', 'Aircraft.Category', 'Longitude', 'Latitude']

aviation_clean_df = aviation_clean_df.drop(columns=cols_to_drop)

aviation_clean_df.isna().mean().sort_values(ascending=False)

Airport.Code               0.434949
Airport.Name               0.408675
Broad.phase.of.flight      0.289587
Publication.Date           0.172997
Engine.Type                0.076889
Purpose.of.flight          0.075626
Report.Status              0.071708
Number.of.Engines          0.068430
Weather.Condition          0.052504
Frac.Fatal.Serious         0.015684
Registration.Number        0.015500
Injury.Severity            0.012491
Country                    0.002754
Location                   0.000639
serious_accident           0.000000
Total.Passengers           0.000000
fatal_accident             0.000000
Event.Id                   0.000000
fatal_or_serious           0.000000
Event.Year                 0.000000
PlaneType                  0.000000
Engine.Type.Clean          0.000000
Weather.Condition.Clean    0.000000
Purpose.of.flight.Clean    0.000000
destroyed                  0.000000
Total.Serious.Injuries     0.000000
Total.Uninjured            0.000000
Total.Minor.Injuries       0

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [77]:
aviation_clean_df.to_csv('data/AviationData_Cleaned.csv', index=False, encoding='utf-8')